In [ ]:
import sys
sys.path.append("../source")

from bokeh.io import output_notebook, show, export_svgs
from bokeh.models import ColumnDataSource, Arrow, OpenHead, NormalHead, VeeHead, Range1d
output_notebook()

from Match_Analytics import Match
from Tracking_Visualization import draw_pitch, plot_sliding_window, play_match
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
import pandas as pd
import pickle
import os
import glob
from operator import itemgetter
working_dir = os.getcwd()

In [ ]:
row = 9
column = 8
fr = 25
# cov_thresh = 0.08 #for 6 regions
cov_thresh = 0.159 #for 8 regions
g = column

In [ ]:
files = glob.glob(f"path_to_data/pj{fr}_ball{fr}_{g}_{g}_{g}/corner_*.bin")
corners = []
for file in files:
    corner = pickle.load(open(file, "rb"))
    corners.append(corner)

In [ ]:
averages = np.zeros((row,column))

for i in range(len(corners)):
    corner = corners[i].to_numpy()
    averages += corner/len(corners)

In [ ]:
av_per_av = np.zeros((row*column, row*column))
av_lst = averages.tolist()
av_lst = [item for sublist in av_lst for item in sublist]

for j in range(row*column):
    for k in range(row*column):
        av_per_av[j,k] = av_lst[j]*av_lst[k]
        if j == k:
            av_per_av[j,k] = 0

In [ ]:
corr_m = np.zeros((row*column, row*column))

for i in range(len(corners)):
    mat = corners[i].to_numpy()
    mat = mat.tolist()
    mat_list = [item for sublist in mat for item in sublist]
    
    for j in range(row*column):
        for k in range(row*column):
            corr_m[j,k] += mat_list[j]*mat_list[k]/len(corners)
            if j==k:
                corr_m[j,k] = 0

In [ ]:
covariance = corr_m-av_per_av

In [ ]:
cov = [item for sublist in covariance for item in sublist]
plt.plot(cov, "o")
plt.ylim([0.10,0.3])

In [ ]:
def get_binary_correlated_plays(tol,rows,columns,J):

    correlated = np.where(J >= tol)
    positions = []

    for i in range(len(correlated[0])):
        positions.append([correlated[0][i], correlated[1][i], J[correlated[0][i], correlated[1][i]]])
    
    positions = sorted(positions, key=itemgetter(2), reverse=True)

    for x in positions:
        del x[2]

    dicc = {}
    mida_matriu_row = rows
    mida_matriu_column = columns

    cont = 0
    for i in range(mida_matriu_row):
        for j in range(mida_matriu_column):
            dicc[cont] = [i,j]
            cont+=1

    pos = []

    for i in range(len(positions)):
        if (dicc[positions[i][0]][0] == rows-1) | (dicc[positions[i][1]][0] == rows-1):
            continue
        else:
            pos.append([dicc[positions[i][0]], dicc[positions[i][1]]])
       
    new_dicc = {}

    for i in range(mida_matriu_row-1):
        for j in range(mida_matriu_row-1):
            for k in range(mida_matriu_row-1):
                for p in range(mida_matriu_row-1):
                    new_dicc[i,j,k,p] = [[i,j], [k,p]]
                    new_dicc[k,p,i,j] = [[i,j], [k,p]]

    lst = []
    for i in range(len(pos)):
        for j in range(len(pos[i])-1):
            lst.append((pos[i][j][0],pos[i][j][1],pos[i][j+1][0],pos[i][j+1][1]))
                    
    
    common_corners = []
    for i in range(len(lst)):
        common_corners.append(new_dicc[lst[i]])

    
    return np.unique(common_corners, axis=0)

In [ ]:
def most_common_binary_plays(play, pj_frame, b_frame, ini, fin, ball):
    
    cov_i = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/cov_ini.npy", "rb"))
    g_i = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/gauss_ini.npy", "rb"))
    pos_i = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/pos_ini.npy", "rb"))
    means_i = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/mean_ini.npy", "rb"))

    cov_f = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/cov_fin.npy", "rb"))
    g_f = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/gauss_fin.npy", "rb"))
    pos_f = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/pos_fin.npy", "rb"))
    means_f = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/mean_fin.npy", "rb"))
    
    values_i, vectors_i = np.linalg.eig(cov_i)
    x_std_i = np.sqrt(values_i[:,0])
    y_std_i = np.sqrt(values_i[:,1])
    angle_i = np.arctan(vectors_i[:,0,1]/vectors_i[:,0,0])

    values_f, vectors_f = np.linalg.eig(cov_f)
    x_std_f = np.sqrt(values_f[:,0])
    y_std_f = np.sqrt(values_f[:,1])
    angle_f = np.arctan(vectors_f[:,0,1]/vectors_f[:,0,0])
    
    pitch = draw_pitch(pattern="stripes_horizontal_vertical", stripes_number = 5, grass_alpha=0.6)
    
    pitch.ellipse(x=means_i[:,0], 
                    y=means_i[:,1], 
                    width=x_std_i, 
                    height=y_std_i,
                    fill_color = None,
                    line_width = 1.3,
                      line_alpha=0.8,
                    line_color = "royalblue",
                    angle = -angle_i)
    
    pitch.ellipse(x=means_f[:,0], 
                    y=means_f[:,1], 
                    width=x_std_f, 
                    height=y_std_f,
                    fill_color = None,
                    line_width = 1.3,
                  line_alpha = 0.8,
                    line_color = "crimson",
                    angle = -angle_f)

    
    for i in range(len(play)):
        pitch.add_layout(Arrow(end=OpenHead(size=5, line_color="black", line_width=0.8), x_start=means_i[:,0][play[:,0]][i],
                         y_start=means_i[:,1][play[:,0]][i], x_end=means_f[:,0][play[:,1]][i], y_end=means_f[:,1][play[:,1]][i],
                              line_width=0.9))

    return show(pitch)

In [ ]:
pos = get_binary_correlated_plays(cov_thresh,row,column,covariance)
print(len(pos))
for i in range(len(pos)):
    print(i+1)
    most_common_binary_plays(pos[i],fr,fr,g,g,g)

In [ ]:
def Get_Arrows(corner):
    corner_np = corner
    arrs = np.where(corner_np == 1)
    arrows = []
    for i in range(len(arrs[0])):
        if i == len(arrs[0])-1:
            continue
        else:
            arrows.append([arrs[0][i], arrs[1][i]])
    return np.array(arrows)


def Get_Ball_Position(ball_row):
    poss = np.where(ball_row == 1)
    pos = np.array([poss[0][0], poss[1][0]])
    position = np.array([poss[1][0]])
    return position

def plot_arrows(pitch, colors, means, n_components, color, lab, x_std, y_std, angle, color_ellipse, label=True, ax=None):
    source = ColumnDataSource(dict(x = means[:,0], y = means[:,1]))
    pitch.ellipse(x=means[:,0], 
                    y=means[:,1], 
                    width=x_std, 
                    height=y_std,
                    fill_color = None,
                    line_width = 1.3,
                    line_color = color_ellipse,
                      line_alpha = 0.8,
                    angle = -angle)
    return pitch

def plot_ball(pitch, color, mean, n_components, lab, x_std, y_std, angle, label=True, ax=None):
    pitch.image_url(url=["pilota.png"], x=mean[0][0], y=mean[0][1], w=2.0, h=2.0)


    
    return pitch
    
    
    

def Plot_MC_Corners(pj_frame, b_frame, ini, fin, ball, corner_number, arrow_color, arrow_head, pos):
    
    cov_i = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/cov_ini.npy", "rb"))
    g_i = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/gauss_ini.npy", "rb"))
    pos_i = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/pos_ini.npy", "rb"))
    means_i = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/mean_ini.npy", "rb"))

    cov_f = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/cov_fin.npy", "rb"))
    g_f = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/gauss_fin.npy", "rb"))
    pos_f = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/pos_fin.npy", "rb"))
    means_f = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/mean_fin.npy", "rb"))
    
    cov_b = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/cov_ball.npy", "rb"))
    g_b = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/gauss_ball.npy", "rb"))
    pos_b = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/pos_ball.npy", "rb"))
    means_b = np.load(open(f"path_to_data/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/mean_ball.npy", "rb"))
    
    corner = pickle.load(open(working_dir+f"/new_corners/{ini+1}x{ini}_fin/new_corner_{corner_number}.bin", "rb"))
#     print(corner)
    
    combinations = []
    for i in range(len(pos)):
        if (corner[pos[i,0,0], pos[i,0,1]] == 1) & (corner[pos[i,1,0], pos[i,1,1]] == 1):
            combinations.append(i+1)
    print(f"I am a combination of {combinations} binary plays")
    ball_pos = Get_Ball_Position(corner[-1:])
    values_i, vectors_i = np.linalg.eig(cov_i)
    x_std_i = np.sqrt(values_i[:,0])
    y_std_i = np.sqrt(values_i[:,1])
    angle_i = np.arctan(vectors_i[:,0,1]/vectors_i[:,0,0])

    values_f, vectors_f = np.linalg.eig(cov_f)
    x_std_f = np.sqrt(values_f[:,0])
    y_std_f = np.sqrt(values_f[:,1])
    angle_f = np.arctan(vectors_f[:,0,1]/vectors_f[:,0,0])
    
    ball_val, ball_vec = np.linalg.eig(cov_b)
    x_std_b = np.sqrt(ball_val[:,0])
    y_std_b = np.sqrt(ball_val[:,1])
    angle_b = np.arctan(ball_vec[:,0,1]/ball_vec[:,0,0])

    arrows = Get_Arrows(corner)
    colors = plt.cm.twilight_shifted(np.linspace(0.1,0.9,ini))
    pitch = draw_pitch(pattern="stripes_horizontal_vertical", stripes_number = 5, grass_alpha = 0.6)
    pitch = plot_arrows(pitch, colors, means_i, ini, "red", "Initial", x_std_i, y_std_i, angle_i, "royalblue")
    colors = plt.cm.twilight_shifted(np.linspace(0.1,0.9,fin))
    pitch = plot_arrows(pitch, colors, means_f, fin, "blue", "Final", x_std_f, y_std_f, angle_f, "crimson")
    pitch = plot_ball(pitch, "yellow", means_b[ball_pos], ball, "Ball", x_std_b[ball_pos], y_std_b[ball_pos], angle_b[ball_pos])
    
    for i in range(len(arrows)): 
        pitch.add_layout(Arrow(end=OpenHead(size=arrow_head, line_color=arrow_color, line_width=0.8), x_start=means_i[:,0][arrows[:,0]][i],
                               y_start=means_i[:,1][arrows[:,0]][i], x_end=means_f[:,0][arrows[:,1]][i], y_end=means_f[:,1][arrows[:,1]][i], line_color=arrow_color,
                              line_width=0.9))
    
    show(pitch)

In [ ]:
energy_file = f"/new_corners/{row}x{column}_fin/matrix_energies_fin.txt"
with open(os.getcwd()+energy_file, "r") as f:
    E = []
    for line in f.readlines():
        E.append(float(line))

E = np.array(E)
rounded = E.round(decimals=10)
rounded = list(rounded)


a, indexes, counts = np.unique(rounded, return_index=True, return_counts=True)

energies = []
indexs = []
repetitions = []
print(len(indexes))
print(counts)
print(a)
for i in range(len(counts)):
    if counts[i]>=5:
        energies.append(a[i])
        indexs.append(indexes[i])
        repetitions.append(counts[i])



print(repetitions)
print(energies)
print(len(indexs))
indexs

In [ ]:
arrow_color = "black"
size_arrow_head = 5

for i in range(len(indexs)):
    print(E[indexs[i]], repetitions[i], i+1)
    corner = indexs[i]+1
    Plot_MC_Corners(fr,fr,g,g,g,corner,arrow_color,size_arrow_head,pos)